# Blockchain and Cryptocurrencies

This notebook summarizes the key concepts related to blockchain and cryptocurrencies. It is mostly written as notes to myself, in an attempt to consolidate the information found here and there on the web. The objective being to gain an understanding of the blockchain technology by experimenting with prototypes of the key features. Code was gathered from many tutorials, and I've tried to give proper credit to my sources.

Topics to be covered

+ signature
+ blockchain structure
+ mining and validation of blocks
+ verification of transactions (double-spend problem)



## Elements of Crypography

This section is a summary of material found in the epub [Practical Crypography](https://cryptobook.nakov.com/) by Svetlin Nakov.
A basic understaning of cryptography is a first necessary step, since the blockchain uses many cryptographic techniques:

+ hash functions for creating message digests of fixed length which uniquely identifies the input message
+ Assymetric encryption, using related public and private keys
+ Digital signatures, which need to be verified

Each topic is next considered in turn.

### Hash Functions

Let's start with a simple test to visualize a hash result, and show how a small change in the input message translate into a radically different hash code. By design, the original message cannot be infered from the hash code. Proof-of-work mining algorithms use hash functions that are more complicated than this simple example, and are intentionally designed to be computationally intensive. 

In [16]:
import hashlib, binascii
message_1 = b'hello world'
message_2 = b'Hello world'
sha_hash_1 = hashlib.sha3_256(message_1)
sha_hash_2 = hashlib.sha3_256(message_2)
w_1 = sha_hash_1.hexdigest()
w_2 = sha_hash_2.hexdigest()
print(message_1, " -> ", w_1)
print(message_2, " -> ", w_2)
print("digest length (bytes): ", sha_hash_1.digest_size, " (ascii): ", len(w_1) )



b'hello world'  ->  644bcc7e564373040999aac89e7622f3ca71fba1d972fd94a31c3bfbf24e3938
b'Hello world'  ->  369183d3786773cef4e56c7b849e7ef5f742867510b676d6b38f8e38a222d8a2
digest length (bytes):  32  (ascii):  64


### Encryption with symmetric key

A symmetric key means that the same key is used for ciphering and deciphering. 
There are three main components to an encryption scheme with symmetric key:
+ an initialization vector (IV) used as initial value in the calculation,
+ a tag or MAC (message authentication code) code used to verify the consistency of the IV, the key and the message,
+ a cipher key,
+ and finally an algorithm to produce a ciphered text, given the plain text, a key and an IV. 
In the Ethereum wallet, the user's private key is encrypted with AES symmetric key algorithm.


In [37]:
import pyaes, pbkdf2, binascii, os, secrets

password = "abcd"
passwordSalt = os.urandom(64)
# generate a 32 bytes key (64 hex digits)
key = pbkdf2.PBKDF2(password, passwordSalt).read(32)
print("AES encryption key: ", binascii.hexlify(key)[1:20], "...")

# initialization vector (256 bits)
iv = secrets.randbits(256)

# cipher a message
message = "Hello World"
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
ciphertext = aes.encrypt(message)
print('Encrypted message:', binascii.hexlify(ciphertext))
print("length (bytes) message: ", len(message), " cipher: ", len(ciphertext))

# decipher...
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
clear_text = aes.decrypt(ciphertext)
print("Original message: ", message, " Clear text: ", clear_text)


AES encryption key:  b'f2c2c7cc732c20c2c64' ...
Encrypted message: b'a57858927b74d2e28c008e'
length (bytes) message:  11  cipher:  11
Original message:  Hello World  Clear text:  b'Hello World'


### Sharing a symmetric key

Two users, each one with its own public key, can generate a shared secret key (to be used for symmetric encryption) as follows:

In [23]:
import pyDHE

alice = pyDHE.new()
alicePubKey = alice.getPublicKey()
print("alice public key: ", hex(alicePubKey)[1:20], "...")

bob = pyDHE.new()
bobPubKey = bob.getPublicKey()
print("bob public key:   ", hex(bobPubKey)[1:20], "...")

aliceSharedKey = alice.update(bobPubKey)
bobSharedKey = bob.update(alicePubKey)

print("alice shared key: ", hex(aliceSharedKey)[1:20], "...")
print("bob shared key:   ", hex(bobSharedKey)[1:20], "...")





alice public key:  x806ddf0d2356281353 ...
bob public key:    xf89542adb62443c58a ...
alice shared key:  x2d83149ceaa0f5c25a ...
bob shared key:    x2d83149ceaa0f5c25a ...


### Encryption with asymmetric key

This is an important technology for the blockchain, as it provides both an encryption algorithm and a scheme for digital signature of records. Work is divided up between the two keys: messages are encrypted with the public key, and decrypted by the corresponding private key. Digital signature is performed with the private key, and can be verified with the public key. 
The process of encrypting/decrypting a file goes as follows:

1. The file is encrypted with a symmetric key (DEM block)
2. The symmetric key is encrypted with the sender's public key (KEM block)
3. Both blocks are transmitted
4. The receiver decrypts the KEM block with the private key, obtaining the symmetric key
5. The DEM block is finally decrypted with the symmetric key

We won't dig into the theory, but simply experiment with a few examples. An implementation of the steps outlined above is provided in the [pycryptodome documentation](https://pycryptodome.readthedocs.io/en/latest/src/examples.html)




In [9]:
from Crypto.PublicKey import RSA
import io

key = RSA.generate(2048)
pri_key = key.export_key()
pub_key = key.publickey().export_key()
# simulate writing files with public and private keys
file_private_pem = io.BytesIO()
file_private_pem.write(pri_key)
file_receiver_pem = io.BytesIO()
file_receiver_pem.write(pub_key)


450

RSA is used to perform an asymmetric encryption of 
an AES symmetric key. This key is used to encrypt the data itself.

In [15]:
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP

data = "We shall attack at dawn".encode("utf-8")

recipient_key = RSA.import_key(file_receiver_pem.getvalue())
session_key = get_random_bytes(16)

# Encrypt the session key with the public RSA key
cipher_rsa = PKCS1_OAEP.new(recipient_key)
enc_session_key = cipher_rsa.encrypt(session_key)

# Encrypt the data with the AES session key
cipher_aes = AES.new(session_key, AES.MODE_EAX)
ciphertext, tag = cipher_aes.encrypt_and_digest(data)

# simulate writing the encrypted file
file_encrypted_data = io.BytesIO()
[file_encrypted_data.write(x) for x in(enc_session_key, cipher_aes.nonce, tag, ciphertext)]


0

The receiver first deciphers the session key, then the message itself.

In [41]:
private_key = RSA.import_key(file_private_pem.getvalue())

file_encrypted_data.seek(0)
enc_session_key, nonce, tag, ciphertext = \
   [ file_encrypted_data.read(x) for x in (private_key.size_in_bytes(), 16, 16, -1) ]

# Decrypt the session key with the private RSA key
cipher_rsa = PKCS1_OAEP.new(private_key)
session_key = cipher_rsa.decrypt(enc_session_key)

# Decrypt the data with the AES session key to recover the message
cipher_aes = AES.new(session_key, AES.MODE_EAX, nonce)
data = cipher_aes.decrypt_and_verify(ciphertext, tag)

print("deciphered message:", data.decode("utf-8"))


deciphered message: We shall attack at dawn


A more streamlined example is provided by [nakov](https://cryptobook.nakov.com/asymmetric-key-ciphers/rsa-encrypt-decrypt-examples). First generate a pair of keys:

In [33]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import binascii

keyPair = RSA.generate(3072)

Public key:
 n=xc2a9c7a0d232aaf60570cbd56eab ... 
 e=0x10001
----BEGIN PUBLIC KEY-----
MIIBojANBgkqhkiG9w0BAQEFAAOCAY8AMIIBigKCAYEAwqnHoNIyqvYFcMvVbquA
CIyMf09m


The pair being made of a public and a private key:

In [36]:
pubKey = keyPair.publickey()
print(f"Public key:\n n={hex(pubKey.n)[1:30]} ... \n e={hex(pubKey.e)}")
pubKeyPEM = pubKey.exportKey()
print(pubKeyPEM.decode('ascii')[1:100])

Public key:
 n=xc2a9c7a0d232aaf60570cbd56eab ... 
 e=0x10001
----BEGIN PUBLIC KEY-----
MIIBojANBgkqhkiG9w0BAQEFAAOCAY8AMIIBigKCAYEAwqnHoNIyqvYFcMvVbquA
CIyMf09m


In [37]:
print(f"Private key:\n n={hex(pubKey.n)[1:30]} ... \n d={hex(keyPair.d)[1:30]} ...")
privKeyPEM = keyPair.exportKey()
print(privKeyPEM.decode('ascii')[1:100])


Private key:
 n=xc2a9c7a0d232aaf60570cbd56eab ... 
 d=x3c06c0a12dd023f9c349ba0131a4 ...
----BEGIN RSA PRIVATE KEY-----
MIIG4gIBAAKCAYEAwqnHoNIyqvYFcMvVbquACIyMf09mTt9OkyPtj/f6Pjoxuf0n
H+3


The next step is to encrypt the message (steps 1 and 2 from the above discussion)

In [39]:
msg = b'We shall attack at dawn'
encryptor = PKCS1_OAEP.new(pubKey)
encrypted = encryptor.encrypt(msg)
print("Encrypted:", binascii.hexlify(encrypted)[1:100], "...")

Encrypted: b'7f63bf59d048dca242063a9c6eefa5dc35c15c7cd4d24fd54d6e1312be87c1435e4f12d3efa997e26b6e57b38d4f46fa8fa' ...


The decryption follows

In [40]:
decryptor = PKCS1_OAEP.new(keyPair)
decrypted = decryptor.decrypt(encrypted)
print('Decrypted:', decrypted)

Decrypted: b'We shall attack at dawn'


## Signature


The signature of a message is the encryption, with the signer's private key, of the hash of the message. Anyone with the signer's public key can therefore verify a signature by performaing two calculations:

1. hash the message
2. decrypt the signature with the public key

The hash code of the message must be identical to the decrypted signature. The following example is taken from the [pycryptodome documentation](https://pycryptodome.readthedocs.io/en/latest/src/signature/dsa.html?highlight=signature), using the same public/private key pair as in the previous section:

In [ ]:
from Crypto.Hash import SHA256
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS

message = b'I give my permission to order #4355'

file_encrypted_data.seek(0)

key = ECC.import_key(open('privkey.der').read())
>>> h = SHA256.new(message)
>>> signer = DSS.new(key, 'fips-186-3')
>>> signature = signer.sign(h)

# The Blockchain

We start with a highly simplified model of blockchain described by [Michael Chrupcala](https://medium.com/coinmonks/python-tutorial-build-a-blockchain-713c706f6531)